## M365 module setup






In [1]:
storage_account = 'steduanalytics__update_this'
use_test_env = True

In [2]:
if use_test_env:
    stage1 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage1'
    stage2 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage2'
    stage3 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage3'
else:
    stage1 = 'abfss://stage1@' + storage_account + '.dfs.core.windows.net'
    stage2 = 'abfss://stage2@' + storage_account + '.dfs.core.windows.net'
    stage3 = 'abfss://stage3@' + storage_account + '.dfs.core.windows.net'

In [3]:
stage1_m365 = stage1 + '/m365/DIPData'
stage1_m365_activity = stage1 + '/m365/DIPData/Activity/ApplicationUsage'

# Process Roster data from stage 1 to stage 2
#
# Sets up the edu_dl (stage 2 data lake) with whatever data is found in the DIP inbound folder.
# This includes:
# - adding column names
# - casting values into a schema

# Calendar
df = spark.read.csv(stage1_m365 + '/Roster/Calendar.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'Calendar')
  df = spark.sql("select _c0 Id, _c1 Name, _c2 Description, cast(_c3 as int) SchoolYear, cast(_c4 as boolean) IsCurrent, _c5 ExternalId, to_timestamp(_c6, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c7, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c8 as boolean) IsActive, _c9 OrgId from Calendar")
  df.write.format("parquet").mode("overwrite").save(stage2 + '/m365/Calendar')
# Course
df = spark.read.csv(stage1_m365 + '/Roster/Course.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'Course')
  df = spark.sql("select _c0 Id, _c1 Name, _c2 Code, _c3 Description, _c4 ExternalId, to_timestamp(_c5, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c6, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c7 as boolean) IsActive, _c8 CalendarId from Course")
  df.write.format("parquet").mode("overwrite").save(stage2 + '/m365/Course')
# Org
df = spark.read.csv(stage1_m365 + '/Roster/Org.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'Org')
  df = spark.sql("select _c0 Id, _c1 Name, _c2 Identifier, _c3 ExternalId, to_timestamp(_c4, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c5, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c6 as boolean) IsActive, _c7 ParentOrgId, _c8 RefOrgTypeId, _c9 SourceSystemId from Org")
  df.write.format('parquet').mode('overwrite').save(stage2 + '/m365/Org')
# Person
df = spark.read.csv(stage1_m365 + '/Roster/Person.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'Person')
  df_Person = spark.sql("select _c0 Id, _c1 FirstName, _c2 MiddleName, _c3 LastName, _c4 GenerationCode, _c5 Prefix, _c6 EnabledUser, _c7 ExternalId, to_timestamp(_c8, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c9, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c10 as boolean) IsActive, _c11 SourceSystemId from Person")
  df_Person.write.format('parquet').mode('overwrite').save(stage2 + '/m365/Person')
# PersonIdentifier
df = spark.read.csv(stage1_m365 + '/Roster/PersonIdentifier.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'PersonIdentifier')
  df = spark.sql("select _c0 Id, _c1 Identifier, _c2 Description, _c3 RefIdentifierTypeId, _c4 ExternalId, to_timestamp(_c5, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c6, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c7 as boolean) IsActive, _c8 PersonId, _c9 SourceSystemId from PersonIdentifier")
  df.write.format('parquet').mode('overwrite').save(stage2 + '/m365/PersonIdentifier')
# RefDefinition
df = spark.read.csv(stage1_m365 + '/Roster/RefDefinition.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'RefDefinition')
  df = spark.sql("select _c0 Id, _c1 RefType, _c2 Namespace, _c3 Code, cast(_c4 as int) SortOrder, _c5 Description, cast(_c6 as boolean) IsActive from RefDefinition")
  df.write.format('parquet').mode('overwrite').save(stage2 + '/m365/RefDefinition')
# Section
df = spark.read.csv(stage1_m365 + '/Roster/Section.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'Section')
  df = spark.sql("select _c0 Id, _c1 Name, _c2 Code, _c3 Location, _c4 ExternalId, to_timestamp(_c5, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c6, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c7 as boolean) IsActive, _c8 CourseId, _c9 RefSectionTypeId, _c10 SessionId, _c11 OrgId from Section")
  df.write.format('parquet').mode('overwrite').save(stage2 + '/m365/Section')
# Session
df = spark.read.csv(stage1_m365 + '/Roster/Session.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'Session')
  df = spark.sql("select _c0 Id, _c1 Name, to_timestamp(_c2, 'MM/dd/yyyy hh:mm:ss a') BeginDate, to_timestamp(_c3, 'MM/dd/yyyy hh:mm:ss a') EndDate, _c4 ExternalId, to_timestamp(_c5, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c6, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c7 as boolean) IsActive, _c8 CalendarId, _c9 ParentSessionId, _c10 RefSessionTypeId from Session")
  df.write.format('parquet').mode('overwrite').save(stage2 + '/m365/Session')
# StaffOrgAffiliation
df = spark.read.csv(stage1_m365 + '/Roster/StaffOrgAffiliation.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'StaffOrgAffiliation')
  df = spark.sql("select _c0 Id, cast(_c1 as boolean) IsPrimary, to_timestamp(_c2, 'MM/dd/yyyy hh:mm:ss a') EntryDate, to_timestamp(_c3, 'MM/dd/yyyy hh:mm:ss a') ExitDate, _c4 ExternalId, to_timestamp(_c5, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c6, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c7 as boolean) IsActive, _c8 OrgId, _c9 PersonId, _c10 RefStaffOrgRoleId from StaffOrgAffiliation")
  df.write.format('parquet').mode('overwrite').save(stage2 + '/m365/StaffOrgAffiliation')
# StaffSectionMembership
df = spark.read.csv(stage1_m365 + '/Roster/StaffSectionMembership.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'StaffSectionMembership')
  df = spark.sql("select _c0 Id, cast(_c1 as boolean) IsPrimaryStaffForSection, to_timestamp(_c2, 'MM/dd/yyyy hh:mm:ss a') EntryDate, to_timestamp(_c3, 'MM/dd/yyyy hh:mm:ss a') ExitDate, _c4 ExternalId, to_timestamp(_c5, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c6, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c7 as boolean) IsActive, _c8 PersonId, _c9 RefStaffSectionRoleId, _c10 SectionId from StaffSectionMembership")
  df.write.format('parquet').mode('overwrite').save(stage2 + '/m365/StaffSectionMembership')
# StudentOrgAffiliation
df = spark.read.csv(stage1_m365 + '/Roster/StudentOrgAffiliation.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'StudentOrgAffiliation')
  df = spark.sql("select _c0 Id, cast(_c1 as boolean) IsPrimary, to_timestamp(_c2, 'MM/dd/yyyy hh:mm:ss a') EntryDate, to_timestamp(_c3, 'MM/dd/yyyy hh:mm:ss a') ExitDate, _c4 ExternalId, to_timestamp(_c5, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c6, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c7 as boolean) IsActive, _c8 OrgId, _c9 PersonId, _c10 RefGradeLevelId, _c11 RefStudentOrgRoleId, _c12 RefEnrollmentStatusId from StudentOrgAffiliation")
  df.write.format('parquet').mode('overwrite').save(stage2 + '/m365/StudentOrgAffiliation')
# StudentSectionMembership
df = spark.read.csv(stage1_m365 + '/Roster/StudentSectionMembership.csv', header='false')
if (df.count() > 0):
  sqlContext.registerDataFrameAsTable(df, 'StudentSectionMembership')
  df = spark.sql("select _c0 Id, to_timestamp(_c1, 'MM/dd/yyyy hh:mm:ss a') EntryDate, to_timestamp(_c2, 'MM/dd/yyyy hh:mm:ss a') ExitDate, _c3 ExternalId, to_timestamp(_c4, 'MM/dd/yyyy hh:mm:ss a') CreateDate, to_timestamp(_c5, 'MM/dd/yyyy hh:mm:ss a') LastModifiedDate, cast(_c6 as boolean) IsActive, _c7 PersonId, _c8 RefGradeLevelWhenCourseTakenId, _c9 RefStudentSectionRoleId, _c10 SectionId from StudentSectionMembership")
  df.write.format('parquet').mode('overwrite').save(stage2 + '/m365/StudentSectionMembership')

In [4]:
# Process Activity data from stage1 into stage2.
#
# If this is the first load, it loads all activity data.
# If this is a subsequent load, it determines the max date currently stored and only loads data from after that date.

def append_to_activity_table(max_date=False):
    df = spark.read.csv(stage1_m365_activity, header='false') 
    sqlContext.registerDataFrameAsTable(df, 'Activity')
    df_Activity = spark.sql("select to_timestamp(_c0) BinDate, _c1 Upn, _c2 UserId, _c3 Application, cast(_c4 as int) SumNumberOfSignals, _c5 Client, cast(_c6 as int) Duration, _c7 LearningActivity, '' PersonId from Activity")
    
    if (max_date):
        df_Activity = df_Activity.filter(df_Activity.BinDate > max_date)

    if (df_Activity.count() == 0):
        print('No new activity data to load')
    else:
        print('Adding activity data later than: ' + str(max_date))
        # The assumption here is that there will always be data in these inbound files
        sqlContext.registerDataFrameAsTable(df_Activity, 'Activity')
        sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(stage2 + '/m365/PersonIdentifier'), 'PersonIdentifier')
        sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(stage2 + '/m365/RefDefinition'), 'RefDefinition')

        df1 = spark.sql( \
        "select act.BinDate, act.Upn, act.UserId, act.Application, act.SumNumberOfSignals, act.Client, act.Duration, act.LearningActivity, pi.PersonId \
        from PersonIdentifier pi, RefDefinition rd, Activity act \
        where \
            pi.RefIdentifierTypeId = rd.Id \
            and rd.RefType = 'RefIdentifierType' \
            and rd.Code = 'username' \
            and pi.Identifier = act.Upn \
            and act.Upn <> '' \
        ")

        df2 = spark.sql( \
        "select act.BinDate, act.Upn, act.UserId, act.Application, act.SumNumberOfSignals, act.Client, act.Duration, act.LearningActivity, pi.PersonId \
        from PersonIdentifier pi, RefDefinition rd, Activity act\
        where \
            pi.RefIdentifierTypeId = rd.Id\
            and rd.RefType = 'RefIdentifierType'\
            and rd.Code = 'ActiveDirectoryId'\
            and pi.Identifier = act.UserId\
            and act.UserId is not null\
        ")

        df1.write.format("parquet").mode("append").save(stage2 + '/m365/Activity')
        df2.write.format("parquet").mode("append").save(stage2 + '/m365/Activity')

try:
    df = spark.read.format('parquet').load(stage2 + '/m365/Activity')
    sqlContext.registerDataFrameAsTable(df, 'Activity')
    # Bad data with a date in the future can prevent the uploading of new activity data, therefore ensure that the watermark is calculated on good data by filtering with CURRENT_TIMESTAMP
    df1 = spark.sql("select BinDate from Activity where BinDate < CURRENT_TIMESTAMP")
    max_date = df1.agg({'BinDate': 'max'}).first()[0]
    print(max_date)
    append_to_activity_table(max_date)    
except:
    print("No Activity data has been loaded into stage2 data lake yet.")
    append_to_activity_table()



2020-10-29 13:56:09
No new activity data to load

In [6]:
# Anonymize the data from edu_dl (stage2) and load into anon_edu_dl (stage3)
# - redact columns from Person table
# - apply a hash to every occurrence of PersonId
# - redact UPN and UserId from Activity table
# - don't bring in PersonIdentifier, Student, Staff and other tables not needed or empty (some tables are not being populated by EDP)

from pyspark.sql.functions import sha2, lit
# Activity
df = spark.read.format('parquet').load(stage2 + '/m365/Activity')
df = df.withColumn('PersonId', sha2(df.PersonId, 256)).withColumn('Upn', lit('*')).withColumn('UserId', lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/m365/Activity')
# Calendar, Course, Org
spark.read.format('parquet').load(stage2 + '/m365/Calendar').write.format('parquet').mode('overwrite').save(stage3 + '/m365/Calendar')
spark.read.format('parquet').load(stage2 + '/m365/Course').write.format('parquet').mode('overwrite').save(stage3 + '/m365/Course')
spark.read.format('parquet').load(stage2 + '/m365/Org').write.format('parquet').mode('overwrite').save(stage3 + '/m365/Org')
# Person
df = spark.read.format('parquet').load(stage2 + '/m365/Person')
df = df.withColumn('Id', sha2(df.Id, 256)).withColumn('FirstName', lit('*')).withColumn("MiddleName", lit('*')).withColumn('LastName', lit('*')).withColumn('ExternalId', sha2(df.ExternalId, 256))
df.write.format('parquet').mode('overwrite').save(stage3 + '/m365/Person')
# PersonIdentifier
df = spark.read.format('parquet').load(stage2 + '/m365/PersonIdentifier')
df = df.withColumn('PersonId', sha2(df.Id, 256)).withColumn('Identifier', lit('*')).withColumn("ExternalId", lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/m365/PersonIdentifier')
# RefDefinition, Section, Session
spark.read.format('parquet').load(stage2 + '/m365/RefDefinition').write.format('parquet').mode('overwrite').save(stage3 + '/m365/RefDefinition')
spark.read.format('parquet').load(stage2 + '/m365/Section').write.format('parquet').mode('overwrite').save(stage3 + '/m365/Section')
spark.read.format('parquet').load(stage2 + '/m365/Session').write.format('parquet').mode('overwrite').save(stage3 + '/m365/Session')
# StaffOrgAffiliation
df = spark.read.format('parquet').load(stage2 + '/m365/StaffOrgAffiliation')
df = df.withColumn('PersonId', sha2(df.PersonId, 256)).withColumn('ExternalId', lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/m365/StaffOrgAffiliation')
# StaffSectionMembership
df = spark.read.format('parquet').load(stage2 + '/m365/StaffSectionMembership')
df = df.withColumn('PersonId', sha2(df.PersonId, 256)).withColumn('ExternalId', lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/m365/StaffSectionMembership')
# StudentOrgAffiliation
df = spark.read.format('parquet').load(stage2 + '/m365/StudentOrgAffiliation')
df = df.withColumn('PersonId', sha2(df.PersonId, 256)).withColumn('ExternalId', lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/m365/StudentOrgAffiliation')
# StudentSectionMembership
df = spark.read.format('parquet').load(stage2 + '/m365/StudentSectionMembership')
df = df.withColumn('PersonId', sha2(df.PersonId, 256)).withColumn('ExternalId', lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/m365/StudentSectionMembership')

In [7]:
# Create spark db to allow for access to the data in the delta-lake via SQL on-demand.
# This is only creating metadata for SQL on-demand, pointing to the data in the delta-lake.
# This also makes it possible to connect in Power BI via the azure sql data source connector.
def create_spark_db(db_name, source_path):
    spark.sql('CREATE DATABASE IF NOT EXISTS ' + db_name)
    spark.sql("create table if not exists " + db_name + ".Activity using PARQUET location '" + source_path + "/Activity'")
    spark.sql("create table if not exists " + db_name + ".Calendar using PARQUET location '" + source_path + "/Calendar'")
    spark.sql("create table if not exists " + db_name + ".Course using PARQUET location '" + source_path + "/Course'")
    spark.sql("create table if not exists " + db_name + ".Org using PARQUET location '" + source_path + "/Org'")
    spark.sql("create table if not exists " + db_name + ".Person using PARQUET location '" + source_path + "/Person'")
    spark.sql("create table if not exists " + db_name + ".PersonIdentifier using PARQUET location '" + source_path + "/PersonIdentifier'")
    spark.sql("create table if not exists " + db_name + ".RefDefinition using PARQUET location '" + source_path + "/RefDefinition'")
    spark.sql("create table if not exists " + db_name + ".Section using PARQUET location '" + source_path + "/Section'")
    spark.sql("create table if not exists " + db_name + ".Session using PARQUET location '" + source_path + "/Session'")
    spark.sql("create table if not exists " + db_name + ".StaffOrgAffiliation using PARQUET location '" + source_path + "/StaffOrgAffiliation'")
    spark.sql("create table if not exists " + db_name + ".StaffSectionMembership using PARQUET location '" + source_path + "/StaffSectionMembership'")
    spark.sql("create table if not exists " + db_name + ".StudentOrgAffiliation using PARQUET location '" + source_path + "/StudentOrgAffiliation'")
    spark.sql("create table if not exists " + db_name + ".StudentSectionMembership using PARQUET location '" + source_path + "/StudentSectionMembership'")

db_prefix = 'test_' if use_test_env else ''
create_spark_db(db_prefix + 's2_m365', stage2 + '/m365')
create_spark_db(db_prefix + 's3_m365', stage3 + '/m365')